In [1]:
import functions 

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


### Pre Process

In [3]:
df = pd.read_parquet("files\\antennas.parquet")

# Create a new DataFrame 'df_RSU' with unique 'X' and 'Y' values by dropping duplicates and copying the result
df_RSU = df[['X', 'Y']].drop_duplicates().copy()

# Sort the 'df_RSU' DataFrame by the 'X' and 'Y' columns in ascending order
df_RSU.sort_values( by = ['X', 'Y'], 
                    inplace = True)

# Reset the index of 'df_RSU' and drop the old index
df_RSU.reset_index( drop = True, 
                    inplace = True)

# Add a new column 'IDR' to 'df_RSU' containing index values
df_RSU["IDR"] = list(df_RSU.index)

# Merge 'df_RSU' into the original DataFrame 'df' using a left join
df = df.merge(df_RSU, how = "left")

# Create a new DataFrame 'df_total_time' by grouping 'ID' and summing 'Permanence (s)' within each group
df_total_time = df[["ID", "Permanence (s)"]].groupby("ID").sum().reset_index()

# Rename the 'Permanence (s)' column in 'df_total_time' to 'Total Time'
df_total_time.rename(columns = {"Permanence (s)" : "Total Time"}, inplace = True)

# Merge 'df_total_time' into the original DataFrame 'df' using a left join
df = df.merge(df_total_time, how = "left")

# Calculate a new 'Permanence (%)' column by dividing 'Permanence (s)' by 'Total Time'
df["Permanence (%)"] = df["Permanence (s)"] / df["Total Time"]

In [4]:
# Create an empty dictionary 'permanence_dict' to store calculated values
permanence_dict = {}

# Loop through each row in the DataFrame 'df' using the iterrows() function
for i in df.iterrows():
    #print(f"{i[0]} de {df.shape[0]}")

    # Assign the 'ID' and 'IDR' values from the current row as a tuple
    # and store the corresponding 'Permanence (%)' value in the dictionary
    permanence_dict[i[1]["ID"], i[1]["IDR"]] = i[1]["Permanence (%)"]

In [5]:
# Saves the new df in parquet
df.to_parquet("files\\vehicles.parquet")

# Saves the new df in parquet
df_RSU.to_parquet("files\\antennas.parquet")

# Saves the matrix remains in pickle
functions.save_pickle(permanence_dict, "files\\permanence_dict.pkl")